In [1]:
"""
@author: Zhen Tong
"""
import rawpy
import os
import sys
import time
import argparse
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"
import cv2 #only for save images to jpg
import exifread
from matplotlib import pyplot as plt #use it only for debug
import numpy as np
from tqdm import tqdm

from isp.BayerDomainProcessor import deadPixelCorrection, blackLevelCompensation, \
    antiAliasingFilter, AWB, CFA_Interpolation
def get_arg():
    parser = argparse.ArgumentParser()
    parser.add_argument("--draw-intermediate", type=bool, default=True)
    parser.add_argument("--set-dir", type=str, default="../data/set03")
    parser.add_argument("--output-dir", type=str, default="data")
    parser.add_argument("--itmd_dir", type=str, default="itmd", help=\
        "intermediate file root directory.")
    args = parser.parse_args()
    assert os.path.exists(args.set_dir)
    if os.path.exists(args.output_dir) == False:
        os.makedirs(args.output_dir)

    args.itmd_dir = os.path.join(args.itmd_dir, args.set_dir.strip("/")[-1])
    if os.path.exists(args.itmd_dir) == False:
        os.makedirs(args.itmd_dir)
    assert os.path.exists(args.output_dir)
    assert os.path.exists(args.itmd_dir)
    return args


In [15]:
# fake args
class ARGs:
    def __init__(self):
        self.set_dir = "../../data/set03"
        self.draw_intermediate = True
        self.output_dir = "../data"
        self.itmd_dir = "../itmd"
        self.itmd_dir = os.path.join(self.itmd_dir, self.set_dir.strip("/")[-1])
        if os.path.exists(self.itmd_dir) == False:
            os.makedirs(self.itmd_dir)
        assert os.path.exists(self.output_dir)
        assert os.path.exists(self.itmd_dir)

In [16]:
args = ARGs()
image_name_list = []
for file in os.listdir(args.set_dir):
    if os.path.splitext(file)[1]=='.ARW': 
        file_dir = os.path.join(args.set_dir, file)
        image_name_list.append(file_dir)
        assert os.path.exists(file_dir)

image_name_list.sort()
image_name_list
print(image_name_list)

camera_white_level_per_channel_list = []
image_list = []
exposure_times = []
for name in image_name_list:
    raw = rawpy.imread(name)
    image_list.append(raw.raw_image)
    camera_white_level_per_channel_list.append(raw.camera_white_level_per_channel[0])
    raw_file = open(name, 'rb')
    exif_file = exifread.process_file(raw_file, details=False, strict=True)
    ISO_str = exif_file['EXIF ISOSpeedRatings'].printable
    ExposureTime = exif_file['EXIF ExposureTime'].printable
    exposure_times.append(float(ExposureTime.split(
        '/')[0]) / float(ExposureTime.split('/')[1]) * float(ISO_str))
    rawimg = raw.raw_image

clip_list = []
for image in image_list:
    clip_list.append(np.clip(image, raw.black_level_per_channel[0], 2**14))
print(50*'-' + '\nLoading {} RAW images Done......'.format(len(clip_list)))


['../../data/set03/DSC00163.ARW', '../../data/set03/DSC00164.ARW', '../../data/set03/DSC00165.ARW', '../../data/set03/DSC00166.ARW', '../../data/set03/DSC00167.ARW', '../../data/set03/DSC00168.ARW', '../../data/set03/DSC00169.ARW']
--------------------------------------------------
Loading 7 RAW images Done......


In [17]:
#####################################  Part 1: Bayer Domain Processing Steps  ###################################
# Step 1. Dead Pixel Correction (10pts)
Bayer_dpcs = []
dpc_thres, dpc_mode = 200, 'gradient'
for i, raw_image in tqdm(enumerate(clip_list)):
    dpc_clip = camera_white_level_per_channel_list[i]
    dpc = deadPixelCorrection(raw_image, dpc_thres, dpc_mode, dpc_clip)
    Bayer_dpcs.append(dpc.execute())
print(50*'-' + '\n 1.1 Dead Pixel Correction Done......')
if args.draw_intermediate:
    cv2.imwrite(os.path.join(args.itmd_dir,'Dead Pixel Correction.jpg'),Bayer_dpcs[0])


1it [00:21, 21.24s/it]

: 